In [1]:
import pandas as pd
import matplotlib.pylab as plt
from matplotlib import font_manager,rc
from selenium import webdriver
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import platform
import seaborn as sns
import sys
import time
import re

if platform.system()=='Windows':
    font_path='c:/Windows/Fonts/malgun.ttf'

In [ ]:
## 스타벅스 매장 입지에 대한 2가지 가설
# 1. 거주 인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
# 2. 직장인이 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.

In [ ]:
### 데이터 수집 (서울 내)
# 1. 스타벅스들의 위치
# 2. 인구 통계 데이터 수집
 

In [4]:
## 스타벅스 매장 목록 데이터 생성
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
#driver
driver=webdriver.Chrome('C:\webdriver\chromedriver.exe')
url='https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

C:\Users\djagk\AppData\Local\Temp\ipykernel_13724\3513795500.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('C:\webdriver\chromedriver.exe')


In [5]:
seoul_btn='.sido_arae_box>li:nth-child(1)>a'
driver.find_element_by_css_selector(seoul_btn).click()


C:\Users\djagk\AppData\Local\Temp\ipykernel_13724\3861491626.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(seoul_btn).click()


In [6]:
all_btn='.gugun_arae_box>li:nth-child(1)>a'
driver.find_element_by_css_selector(all_btn).click()

C:\Users\djagk\AppData\Local\Temp\ipykernel_13724\1038311480.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(all_btn).click()


In [ ]:
html=driver.page_source
soup=BeautifulSoup(html,"html.parser")
soup

In [ ]:
starbucks_store_list=soup.select('li.quickResultLstCon')
starbucks_store_list

In [171]:
## 매장명, 위도, 경도 매장타입, 주소, 전화번호
all_list=[]
for i in range(len(starbucks_store_list)):
    name=starbucks_store_list[i].find('strong').text.strip()
    addr=starbucks_store_list[i].find('p').contents[0]
    tel=starbucks_store_list[i].find('p').contents[2]
    x=starbucks_store_list[i]['data-lat']
    y=starbucks_store_list[i]['data-long']
    ty=starbucks_store_list[i].find("i")['class'][0][4:]
    list=[name,x,y,ty,addr,tel]
    all_list.append(list)
columns=['매장명','위도','경도','매장타입','주소','전화번호']
starbucks_list=pd.DataFrame(all_list,columns=columns)
starbucks_list.to_excel('data_starbucks/seoul_starbucks_list.xlsx',index=False)
#starbucks_store_list

In [47]:
## 서울 열린 데이터 광장 공공 데이터 수집 및 처리
# 주민등록 인구 통계 데이터
sgg_pop_df=pd.read_csv('data_starbucks/report.txt',sep='\t',header=2,)
columns={
    '기간': 'GIGAN',
    '자치구': 'JACHIGU',
    '계':  'GYE_1',
    '계.1':'GYE_2',
    '계.2':'GYE_3',
    '남자':'NAMJA_1',
    '남자.1':'NAMJA_2',
    '남자.2':'NAMJS_3',
    '여자':'YEOJA_1',
    '여자.1':'YEOJA_2',
    '여자.2':'YEOJA_3',
    '세대':'SEDAE',
    '세대당인구':'SEDAEDANGINGU',
    '65세이상고령자':'N 65SEISANGGORYEONGJA'
}
sgg_pop_df.rename(columns=columns,inplace=True)
sgg_pop_df.head()
#sgg_pop_df.info()

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJS_3,YEOJA_3,SEDAEDANGINGU,N 65SEISANGGORYEONGJA
0,2022.1/4,합계,"4,451,444","9,732,617","4,716,704","5,015,913","9,506,778","4,613,910","4,892,868","225,839","102,794","123,045",2.14,"1,628,980"
1,2022.1/4,종로구,"73,766","153,684","74,018","79,666","144,275","69,911","74,364","9,409","4,107","5,302",1.96,"28,064"
2,2022.1/4,중구,"63,767","131,450","63,869","67,581","122,173","59,448","62,725","9,277","4,421","4,856",1.92,"24,742"
3,2022.1/4,용산구,"111,207","236,518","114,571","121,947","222,169","106,735","115,434","14,349","7,836","6,513",2.00,"39,461"
4,2022.1/4,성동구,"134,533","291,609","141,662","149,947","285,073","138,847","146,226","6,536","2,815","3,721",2.12,"46,896"


In [63]:
### 필요없는 데이터 제거, 합계(자치구 제거)
condition=sgg_pop_df['JACHIGU']!='합계'
sgg_pop_df_selected=sgg_pop_df[condition]
sgg_pop_df_selected
## 분석에 필요한 컬럼
columns=['JACHIGU','GYE_1']
sgg_pop_df_final=sgg_pop_df_selected[columns]
sgg_pop_df_final

#데이터프레임 컬럼명 변경
sgg_pop_df_final.columns=['시군구명','주민등록인구']
sgg_pop_df_final=sgg_pop_df_final.reset_index(drop=True)
sgg_pop_df_final


,시군구명,주민등록인구
0,종로구,"153,684"
1,중구,"131,450"
2,용산구,"236,518"
3,성동구,"291,609"
4,광진구,"353,110"
5,동대문구,"352,592"
6,중랑구,"393,600"
7,성북구,"442,680"
8,강북구,"300,824"
9,도봉구,"318,100"


In [61]:
## 서울시 동별 사업체 현황 통계
sgg_biz_df=pd.read_csv('data_starbucks/report2.txt',sep='\t',header=2)
columns={
    '기간':'GIGAN',
    '자치구':'JACHIGU',
    '동':'DONG',
    '사업체수':'SAEOPCHESU_1',
    '여성대표자':'YEOSEONGDAEPYOJA',
    '계':'GYE',
    '남':'NAM',
    '여':'YEO'
}
sgg_biz_df.rename(columns=columns,inplace=True)
sgg_biz_df
# 시군구별 사업체 현황 데이터 추출 (동별x)
condition=sgg_biz_df['DONG']=='소계'
sgg_biz_df_selected=sgg_biz_df[condition]
sgg_biz_df_selected

## 필요없는 컬럼 제거
columns=['JACHIGU','GYE','SAEOPCHESU_1']
sgg_biz_df_final=sgg_biz_df_selected[columns]
sgg_biz_df_final.columns=['시군구명','종사자수','사업체수']
sgg_biz_df_final

# 인덱스 초기화
sgg_biz_df_final=sgg_biz_df_final.reset_index(drop=True)
sgg_biz_df_final

,시군구명,종사자수,사업체수
0,종로구,"260,446","39,679"
1,중구,"392,568","60,127"
2,용산구,"135,240","20,254"
3,성동구,"178,243","28,343"
4,광진구,"126,305","24,445"
5,동대문구,"138,400","31,324"
6,중랑구,"103,345","28,228"
7,성북구,"111,005","23,617"
8,강북구,"73,904","19,014"
9,도봉구,"72,939","18,628"


In [65]:
# 엑셀로 저장
sgg_pop_df_final.to_excel('data_starbucks/sgg_pop.xlsx',index=False)
sgg_biz_df_final.to_excel('data_starbucks/sgg_biz.xlsx',index=False)

In [ ]:
## 1. 서울시 인구조사 (시군구별) -data_starbucks/sgg_pop.xlsx
## 2. 서울시 사업체 데이터 수집 -data_starbucks/sgg_biz.xlsx
## 3. 스타벅스 매장목록 - data_starbucks/seoul_starbucks_list.xlsx
